## ロジスティック回帰モデルを構築する - レッスン4

![ロジスティック回帰と線形回帰のインフォグラフィック](../../../../../../translated_images/linear-vs-logistic.ba180bf95e7ee66721ba10ebf2dac2666acbd64a88b003c83928712433a13c7d.ja.png)

#### **[講義前のクイズ](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### イントロダクション

回帰に関する最後のレッスンでは、基本的な*クラシック*な機械学習技術の一つであるロジスティック回帰について学びます。この技術を使うことで、パターンを見つけて二値分類を予測することができます。このキャンディはチョコレートかどうか？この病気は伝染性かどうか？この顧客はこの商品を選ぶかどうか？

このレッスンで学ぶ内容:

- ロジスティック回帰の技術

✅ このタイプの回帰を扱う理解を深めるために、この[学習モジュール](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)を参照してください。

## 前提条件

かぼちゃのデータを扱ったことで、`Color`という二値分類が扱えることに気づいているはずです。

いくつかの変数を基にして、*特定のかぼちゃの色が何色である可能性が高いか*（オレンジ 🎃 か白 👻）を予測するロジスティック回帰モデルを構築してみましょう。

> なぜ回帰に関するレッスンで二値分類について話しているのでしょうか？それは言語的な便宜上の理由に過ぎません。ロジスティック回帰は[実際には分類方法](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)であり、線形ベースのものです。次のレッスングループでは、データを分類する他の方法について学びます。

このレッスンでは、以下のパッケージが必要です:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/)は、データサイエンスをより速く、簡単に、そして楽しくするために設計された[Rパッケージのコレクション](https://www.tidyverse.org/packages)です。

- `tidymodels`: [tidymodels](https://www.tidymodels.org/)フレームワークは、モデリングと機械学習のための[パッケージのコレクション](https://www.tidymodels.org/packages/)です。

- `janitor`: [janitorパッケージ](https://github.com/sfirke/janitor)は、汚れたデータを調査し、クリーニングするためのシンプルなツールを提供します。

- `ggbeeswarm`: [ggbeeswarmパッケージ](https://github.com/eclarke/ggbeeswarm)は、ggplot2を使用してビースウォームスタイルのプロットを作成する方法を提供します。

以下のコマンドでこれらをインストールできます:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

または、以下のスクリプトを使用して、必要なパッケージがインストールされているか確認し、欠けている場合はインストールすることもできます。


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **質問を定義する**

今回の目的では、質問を「白」または「白ではない」という二択で表現します。データセットには「縞模様」というカテゴリもありますが、該当するデータが少ないため使用しません。そもそも、データセットから欠損値を除去すると、このカテゴリは消えてしまいます。

> 🎃 面白い事実として、白いカボチャを「ゴースト」カボチャと呼ぶことがあります。彫刻するのが難しいため、オレンジ色のカボチャほど人気はありませんが、見た目はとてもクールです！したがって、質問を「ゴースト」または「ゴーストではない」と再構成することもできます。👻

## **ロジスティック回帰について**

ロジスティック回帰は、以前学んだ線形回帰とはいくつか重要な点で異なります。

#### **二値分類**

ロジスティック回帰は、線形回帰と同じ機能を提供するわけではありません。ロジスティック回帰は「二値カテゴリ」（例:「オレンジかオレンジではない」）について予測を行いますが、線形回帰は「連続値」を予測することができます。例えば、カボチャの産地と収穫時期を基にして、*価格がどれだけ上昇するか*を予測することが可能です。

![Dasani Madipalliによるインフォグラフィック](../../../../../../translated_images/pumpkin-classifier.562771f104ad5436b87d1c67bca02a42a17841133556559325c0a0e348e5b774.ja.png)

### その他の分類

ロジスティック回帰には、他にも多項式回帰や順序回帰といった種類があります。

- **多項式回帰**: 複数のカテゴリがある場合に使用します（例:「オレンジ、白、縞模様」）。

- **順序回帰**: 順序付けされたカテゴリを扱う場合に使用します。例えば、カボチャのサイズを有限のサイズ（mini, sm, med, lg, xl, xxl）で論理的に順序付けする場合に役立ちます。

![多項式回帰 vs 順序回帰](../../../../../../translated_images/multinomial-vs-ordinal.36701b4850e37d86c9dd49f7bef93a2f94dbdb8fe03443eb68f0542f97f28f29.ja.png)

#### **変数が相関している必要はない**

線形回帰が相関の強い変数でより良い結果を出すのに対し、ロジスティック回帰は逆で、変数が必ずしも一致している必要はありません。このデータセットのように、相関がやや弱い場合でも適しています。

#### **大量のクリーンなデータが必要**

ロジスティック回帰は、データが多いほど正確な結果を出します。この小規模なデータセットはこのタスクには最適ではないため、その点を考慮してください。

✅ ロジスティック回帰に適したデータの種類について考えてみましょう。

## 演習 - データを整理する

まず、データを少し整理し、欠損値を削除して特定の列だけを選択します。

1. 次のコードを追加してください:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


新しいデータフレームを確認するには、以下のように[*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html)関数を使用すると便利です。


In [ ]:
pumpkins_select %>% 
  glimpse()


バイナリ分類問題を実際に行うことを確認しましょう:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### 可視化 - カテゴリカルプロット
これまでに、再びパンプキンデータを読み込み、いくつかの変数（Colorを含む）を保持するようにデータセットをクリーンアップしました。ggplotライブラリを使用して、ノートブック内でデータフレームを可視化してみましょう。

ggplotライブラリは、データを視覚化するための便利な方法を提供します。例えば、カテゴリカルプロットで各VarietyやColorごとのデータ分布を比較することができます。

1. geombar関数を使用して、パンプキンデータを使い、各パンプキンカテゴリ（オレンジまたは白）に対して色のマッピングを指定することで、このようなプロットを作成します:


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

データを観察すると、ColorデータがVarietyとどのように関連しているかがわかります。

✅ このカテゴリカルプロットを基に、どのような興味深い探求が考えられますか？


### データ前処理: 特徴量エンコーディング

私たちのカボチャのデータセットには、すべての列に文字列値が含まれています。人間にとってカテゴリカルデータを扱うのは直感的ですが、機械にとってはそうではありません。機械学習アルゴリズムは数値データでうまく機能します。そのため、エンコーディングはデータ前処理の段階で非常に重要なステップとなります。これにより、カテゴリカルデータを数値データに変換し、情報を失うことなく処理できるようになります。適切なエンコーディングは、良いモデルを構築するための鍵となります。

特徴量エンコーディングには、主に以下の2種類のエンコーダがあります：

1. **Ordinalエンコーダ**: 順序変数に適しています。順序変数とは、データが論理的な順序に従うカテゴリカル変数のことです。例えば、データセットの`item_size`列が該当します。このエンコーダは、各カテゴリを数値で表すマッピングを作成します。この数値は、列内でのカテゴリの順序を表します。

2. **カテゴリカルエンコーダ**: 名義変数に適しています。名義変数とは、データが論理的な順序に従わないカテゴリカル変数のことです。データセット内の`item_size`以外のすべての特徴量が該当します。このエンコーディングは「ワンホットエンコーディング」と呼ばれ、各カテゴリがバイナリ列で表されます。エンコードされた変数は、カボチャがその品種に属している場合は1、そうでない場合は0となります。

Tidymodelsにはもう一つ便利なパッケージがあります：[recipes](https://recipes.tidymodels.org/) - データ前処理用のパッケージです。このパッケージを使って、すべての予測子列を整数にエンコードする`recipe`を定義し、`prep`を使って必要な量や統計を推定し、最後に`bake`を使って新しいデータに計算を適用します。

> 通常、recipesはモデリングのための前処理として使用されます。この場合、データセットをモデリングに適した状態にするためにどのステップを適用するべきかを定義します。この場合、`prep`と`bake`を手動で使用してレシピを推定するのではなく、`workflow()`を使用することを**強く推奨**します。この内容についてはすぐに説明します。
>
> しかし、今回はrecipes + prep + bakeを使用して、データセットに適用すべきステップを指定し、前処理済みデータをステップ適用後に抽出する方法を学びます。


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ アイテムサイズ列に対して順序エンコーダーを使用する利点は何ですか？

### 変数間の関係を分析する

データの前処理が完了したので、特徴量とラベルの関係を分析し、特徴量を基にモデルがラベルをどの程度予測できるかの見通しを得ることができます。この種の分析を行う最良の方法は、データをプロットすることです。  
ここでは再び、ggplotのgeom_boxplot_関数を使用して、アイテムサイズ、品種、色の関係をカテゴリカルプロットで視覚化します。データをより良くプロットするために、エンコードされたアイテムサイズ列とエンコードされていない品種列を使用します。


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### スウォームプロットを使う

Colorは二値カテゴリ（WhiteまたはNot）であるため、視覚化には「[特別なアプローチ](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)」が必要です。

`swarm plot`を試して、item_sizeに対するColorの分布を表示してみましょう。

ここでは、[ggbeeswarmパッケージ](https://github.com/eclarke/ggbeeswarm)を使用します。このパッケージは、ggplot2を使ってビースウォームスタイルのプロットを作成するためのメソッドを提供します。ビースウォームプロットは、通常は重なってしまうポイントを隣り合わせに配置することで視覚化する方法です。


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


これで、色の二値カテゴリとサイズの大きなグループとの関係についての理解が深まりました。次に、ロジスティック回帰を使って、特定のカボチャの色を予測する方法を探ってみましょう。

## モデルを構築する

分類モデルで使用する変数を選択し、データをトレーニングセットとテストセットに分割します。Tidymodelsのパッケージである[rsample](https://rsample.tidymodels.org/)は、効率的なデータ分割とリサンプリングのためのインフラを提供します。


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 さあ、トレーニング特徴量をトレーニングラベル（色）に適合させることで、モデルのトレーニングを始めましょう。

まず、データをモデリングに適した状態にするための前処理ステップを指定するレシピを作成します。例えば、カテゴリ変数を整数値にエンコードするなどです。`baked_pumpkins`と同様に、`pumpkins_recipe`を作成しますが、ここでは`prep`や`bake`は行いません。これらはワークフローにまとめられるためです。この後のステップでその詳細を確認できます。

Tidymodelsではロジスティック回帰モデルを指定する方法がいくつかあります。`?logistic_reg()`を参照してください。今回は、デフォルトの`stats::glm()`エンジンを使用してロジスティック回帰モデルを指定します。


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


レシピとモデル仕様が揃ったので、それらを一つのオブジェクトにまとめる方法を見つける必要があります。このオブジェクトは、まずデータを前処理（裏でprep+bakeを実行）、前処理済みデータにモデルを適合させ、さらに必要に応じて後処理を行うことができます。

Tidymodelsでは、この便利なオブジェクトは[`workflow`](https://workflows.tidymodels.org/)と呼ばれ、モデリングの構成要素を効率的に保持します。


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


ワークフローが*指定*された後、モデルは[`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html)関数を使用して`訓練`することができます。ワークフローはレシピを評価し、データを前処理してから訓練を行うため、prepやbakeを手動で実行する必要はありません。


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


モデルのトレーニング中に学習された係数が出力されます。

トレーニングデータを使ってモデルを訓練したので、[parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) を使用してテストデータに対する予測を行うことができます。まず、モデルを使ってテストセットのラベルと各ラベルの確率を予測してみましょう。確率が0.5を超える場合、予測クラスは `WHITE` となり、それ以外の場合は `ORANGE` となります。


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


とても良いですね！これでロジスティック回帰の仕組みについてさらに理解が深まりました。

### 混同行列を使ったより良い理解

各予測値を対応する「正解データ」と比較するだけでは、モデルの予測精度を判断する効率的な方法とは言えません。幸いなことに、Tidymodelsにはさらに便利なツールがあります。それが、パフォーマンス指標を使ってモデルの効果を測定するためのパッケージ [`yardstick`](https://yardstick.tidymodels.org/) です。

分類問題に関連するパフォーマンス指標の1つに、[`混同行列`](https://wikipedia.org/wiki/Confusion_matrix) があります。混同行列は、分類モデルの性能を示すものです。混同行列は、各クラスの例がモデルによって正しく分類された数を集計します。今回のケースでは、オレンジ色のカボチャがオレンジ色として分類された数や、白いカボチャが白として分類された数を示します。また、**誤った**カテゴリに分類された数も表示されます。

yardstickの [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) 関数を使うと、観測されたクラスと予測されたクラスのクロスタブを計算することができます。


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


混同行列を解釈してみましょう。私たちのモデルは、カボチャを2つの二値カテゴリ、`white`（白）と`not-white`（白以外）に分類するよう求められています。

- モデルがカボチャを白と予測し、実際にそのカボチャがカテゴリ「white」に属している場合、それを`true positive`（真陽性）と呼びます。これは左上の数字で示されます。

- モデルがカボチャを白以外と予測し、実際にそのカボチャがカテゴリ「white」に属している場合、それを`false negative`（偽陰性）と呼びます。これは左下の数字で示されます。

- モデルがカボチャを白と予測し、実際にそのカボチャがカテゴリ「not-white」に属している場合、それを`false positive`（偽陽性）と呼びます。これは右上の数字で示されます。

- モデルがカボチャを白以外と予測し、実際にそのカボチャがカテゴリ「not-white」に属している場合、それを`true negative`（真陰性）と呼びます。これは右下の数字で示されます。

| 真実 |
|:-----:|

|               |        |       |
|---------------|--------|-------|
| **予測**      | WHITE | ORANGE |
| WHITE         | TP     | FP    |
| ORANGE        | FN     | TN    |

ご想像の通り、`true positive`と`true negative`の数が多く、`false positive`と`false negative`の数が少ない方が望ましいです。これにより、モデルの性能が良いことを意味します。

混同行列は、分類モデルの性能をより良く評価するための他の指標を生み出すために役立ちます。それらをいくつか見ていきましょう：

🎓 精度（Precision）: `TP/(TP + FP)`  
予測された陽性のうち、実際に陽性である割合として定義されます。別名：[陽性予測値](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value")

🎓 再現率（Recall）: `TP/(TP + FN)`  
実際に陽性であるサンプルのうち、陽性結果の割合として定義されます。別名：`感度（Sensitivity）`

🎓 特異度（Specificity）: `TN/(TN + FP)`  
実際に陰性であるサンプルのうち、陰性結果の割合として定義されます。

🎓 正確度（Accuracy）: `TP + TN/(TP + TN + FP + FN)`  
サンプルに対して正確に予測されたラベルの割合。

🎓 F値（F Measure）:  
精度と再現率の加重平均で、最良が1、最悪が0。

これらの指標を計算してみましょう！


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## このモデルのROC曲線を視覚化する

もう一つの視覚化を行い、いわゆる[`ROC曲線`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)を確認してみましょう:


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC曲線は、分類器の出力を真陽性と偽陽性の観点から視覚化するためによく使用されます。ROC曲線では通常、Y軸に`True Positive Rate`（感度）、X軸に`False Positive Rate`（1-特異度）が表示されます。そのため、曲線の急峻さや、中央の直線と曲線の間の空間が重要です。理想的には、曲線がすぐに上昇し、直線を越える形になります。今回の場合、最初に偽陽性が発生し、その後、曲線が適切に上昇して直線を越えます。

最後に、`yardstick::roc_auc()`を使用して、実際の曲線下面積（AUC）を計算してみましょう。AUCの解釈方法の一つとして、モデルがランダムな正の例をランダムな負の例よりも高くランク付けする確率として捉えることができます。


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


結果は約`0.975`です。AUCは0から1の範囲で表されるため、スコアが高いほど良いモデルと言えます。予測が100%正確なモデルはAUCが1になります。この場合、モデルは*かなり良い*と言えるでしょう。

今後の分類に関するレッスンでは、このスコアを改善する方法（例えば、このケースでは不均衡なデータへの対処など）を学ぶことになります。

## 🚀チャレンジ

ロジスティック回帰にはまだまだ学ぶべきことがたくさんあります！しかし、最良の学習方法は実際に試してみることです。このタイプの分析に適したデータセットを見つけて、それを使ってモデルを構築してみましょう。何を学びましたか？ヒント: [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets)で興味深いデータセットを探してみてください。

## 復習と自己学習

[スタンフォード大学のこの論文](https://web.stanford.edu/~jurafsky/slp3/5.pdf)の最初の数ページを読んで、ロジスティック回帰の実用的な利用例について学びましょう。これまで学んだ回帰タスクの中で、どのタスクがどのタイプの回帰に適しているかを考えてみてください。どちらが最適でしょうか？



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確さが含まれる可能性があります。元の言語で記載された原文が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤認について、当社は一切の責任を負いません。
